In [ ]:
import ML_Coupling.tools as tool
import numpy as np
import subprocess, time

xyz_path = './../ML_coupling/input/nat_dimer_para_x0_y0_z35.xyz'
n_atom, atom_sym, geo = tool.read_xyz(xyz_path, output='regular')
n_atom_frag1 = 18
n_atom_frag2 = n_atom - n_atom_frag1
geo_frag1 = geo[0 : n_atom_frag1]
geo_frag2 = geo[n_atom_frag1 : n_atom]

# shift frag1
geo_frag1_ = np.zeros(geo_frag1.shape)
geo_ = np.zeros(geo.shape)

# specify the scanning range
x_shift = np.arange(0, 4.1, 0.1)
y_shift = np.arange(0, 4.1, 0.1)
z_shift = np.arange(3.4, 6.1, 0.1)
x_rot = np.arange(0.0, 91, 2)
y_rot = np.arange(0.0, 91, 2)
z_rot = np.arange(0.0, 91, 2)

# qchem_input_template = '/home/jingheng/ML_data_set/QC_coupling/raw/nat_dimer/template_input.inp'
# with open(qchem_input_template, 'r') as f:
#     qc_input = f.readlines()
t0 = time.time()

trans_rot = []
for k in z_shift:
    k_ = k * 10
    geo_frag1_[:,2] = geo_frag1[:,2] + k
    for i in x_shift:
        i_ = i * 10
        geo_frag1_[:,0] = geo_frag1[:,0] + i
        # input_path = 'z%02d_x%02d'%(int(k_), int(i_))
#        subprocess.run('mkdir '+ input_path, shell=True)
        for j in y_shift:
            j_ = j * 10
            geo_frag1_[:,1] = geo_frag1[:,1] + j            
            # file_name = '/nat_dimer_para_xrot%03d_yrot%03d_zrot%03d.inp'%(int(i), int(j), int(k))

            for kr in z_rot:
                kr_ = kr / 180 * np.pi
                z_rot_tm = np.array([[np.cos(kr_), -np.sin(kr_), 0],[np.sin(kr_), np.cos(kr_), 0], [0,0,1]])
                for ir in x_rot:
                    ir_ = ir / 180 * np.pi
                    x_rot_tm = np.array([[1,0,0], [0, np.cos(ir_), -np.sin(ir_)],[0, np.sin(ir_), np.cos(ir_)]])
                    # input_path = 'zrot%03d_xrot%03d'%(int(kr), int(ir))
                    for jr in y_rot:
                        jr_ = jr /180 * np.pi
                        y_rot_tm = np.array([[np.cos(jr_), 0, np.sin(jr_)],[0,1,0], [-np.sin(jr_), 0, np.cos(jr_)]])
                        geo_frag1_ = np.einsum('ij,jk,kl,lm->im', geo_frag1_, x_rot_tm, y_rot_tm, z_rot_tm)

                        geo_[0:n_atom_frag1]      = geo_frag1_
                        geo_[n_atom_frag1:n_atom] = geo_frag2
                        # check_bond = tool.check_close_mol(mol1=geo_frag1_, mol2=geo_frag2, atom_sym1=atom_sym[0:n_atom_frag1], atom_sym2=atom_sym[n_atom_frag1:n_atom])
                        # print(check_bond)
                        t = time.time() - t0
                        print(t)
                        if check_bond is False:
                            trans_rot.append(geo_)
                            
            # file_name = '/nat_dimer_para_x%02d_y%02d_z%02d.inp'%(int(i_), int(j_), int(k_))
            # print(input_path+file_name)


# with open(qchem_input_template, 'r') as f:
#     qc_input = f.readlines()                    
#           with open(input_path+file_name, 'w') as f:
#               f.write(''.join(qc_input[0:2]))
#               for l in range(0, n_atom):
#                   f.write('%s %15.8f %15.8f %15.8f\n'%(atom_sym[l], geo_[l][0], geo_[l][1], geo_[l][2]))
#               f.write(''.join(qc_input[2+n_atom:]))
#           with open(input_path+'/input_list', 'a+') as inp:
#               inp.write(file_name[1:-4]+'\n')                                    


In [64]:
from joblib import Parallel, delayed
import numpy
t0 = time.time()
xyz_path = './../ML_coupling/input/nat_dimer_para_x0_y0_z35.xyz'
n_atom, atom_sym, geo = tool.read_xyz(xyz_path, output='regular')
n_atom_frag1 = 18
n_atom_frag2 = n_atom - n_atom_frag1
geo_frag1 = geo[0 : n_atom_frag1]
geo_frag2 = geo[n_atom_frag1 : n_atom]

# shift frag1
geo_frag1_ = np.zeros(geo_frag1.shape)
geo_ = np.zeros(geo.shape)

trans=np.array([(geo_frag1[:,0] +i*0.1,geo_frag1[:,1] +j*0.1,geo_frag1[:,2] +k*0.1) for k in range(0,41) for i in range(0,41) for j in range(0, 41)])
trans_=[np.array(trans[i]).T for i in range(0,len(trans))]

def scan_mix(trans):
    rot=[np.einsum('ij,jk,kl,lm->im', geo_frag1,\
               np.array([[1,0,0], [0, np.cos(i*np.pi/180), -np.sin(i*np.pi/180)],[0, np.sin(i*np.pi/180), np.cos(i*np.pi/180)]]),\
               np.array([[np.cos(j*np.pi/180), 0, np.sin(j*np.pi/180)],[0,1,0], [-np.sin(j*np.pi/180), 0, np.cos(j*np.pi/180)]]),\
               np.array([[np.cos(k*np.pi/180), -np.sin(k*np.pi/180), 0],[np.sin(k*np.pi/180), np.cos(k*np.pi/180), 0], [0,0,1]])) for k in range(0,91) for i in range(0,91) for j in range(0, 91)]
    return np.array(rot)

trans_rot = Parallel(n_jobs=-1)(delayed(test)(i, j, k) for i in range(0,10) for j in range(0,10) for k in range(10))
pick_out = []
for geo1 in trans_rot:
    check_bond = tool.check_close_mol(mol1=geo1, mol2=geo_frag2, atom_sym1=atom_sym[0:n_atom_frag1], atom_sym2=atom_sym[n_atom_frag1:n_atom])
    if check_bond is False:
        pick_out.append(geo1)
        
np.save('trans_rot_geo.npy', trans_rot)
np.save(se


print(np.array(trans_[0]))
t1 = time.time()
print(t1-t0)

In [63]:
print(np.array(trans[0]).T)

# 

In [ ]:
from joblib import Parallel, delayed
def test(a,b,c):
    d = a*b*c
    time.sleep(0.01)
    return d
t0 = time.time()

tl = Parallel(n_jobs=-1)(delayed(test)(i, j, k) for i in range(0,10) for j in range(0,10) for k in range(10))
      
t1 = time.time()
print(t1-t0)

In [65]:
print(tl)